In [16]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm, skew 
from scipy.special import boxcox1p
#inv - inv_boxcox1p(y, 2.5)
boxcox_lambda = 1.6

In [21]:
df = pd.read_csv('../housing.data', sep='\s+', header=None, names=['CRIM', 'ZN', 'INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV'])

In [22]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [23]:
df['MEDV'] = boxcox1p(df['MEDV'], boxcox_lambda)

In [24]:
for i in df.columns[:-1]:
    print(i)
    print(df[i].skew(), df[i].kurt())
    if i != 'CHAS' and (abs(df[i].skew()) > 0.75):
        df[i] = np.log1p(df[i])
        print(df[i].skew(), df[i].kurt())
    
    print('--------------------------')

CRIM
5.223148798243851 37.13050912952203
1.2692005882725572 0.5045535413312598
--------------------------
ZN
2.2256663227354307 4.031510083739155
1.1934513868588792 -0.3978413378913239
--------------------------
INDUS
0.29502156787351164 -1.2335396011495188
--------------------------
CHAS
3.405904172058746 9.638263777819526
--------------------------
NOX
0.7293079225348787 -0.06466713336542629
--------------------------
RM
0.40361213328874385 1.8915003664993404
--------------------------
AGE
-0.5989626398812962 -0.9677155941626912
--------------------------
DIS
1.0117805793009007 0.4879411222443908
0.33156144277059646 -0.9050971244339001
--------------------------
RAD
1.0048146482182057 -0.8672319936034931
0.5327599614480961 -0.8616338868557682
--------------------------
TAX
0.669955941795016 -1.1424079924768082
--------------------------
PTRATIO
-0.8023249268537809 -0.28509138330538875
-1.0029822002725293 0.20447932221716103
--------------------------
B
-2.8903737121414492 7.226817549

In [30]:
num_features = df[['CRIM', 'ZN', 'INDUS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']]

In [48]:
catg_features = pd.get_dummies(df['CHAS'], prefix='CHAS')

In [40]:
y = df['MEDV']

In [55]:
catg_features

,CHAS_0,CHAS_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [56]:
pd.concat(num_features, catg_features, axis=1)

TypeError: concat() got multiple values for argument 'axis'

In [ ]:
pd.DataFrame